In [1]:
import pandas as pd
import numpy as np
from importlib import reload
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, mean_squared_error,r2_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
import argparse
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from collections import Counter
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches
import time
from sklearn.pipeline import  Pipeline, make_pipeline

from transformation import RemoveSkewnessKurtosis, Standardize, ImputeNumeric


# Import Data
# path = '/home/daisy/FDA_Dataset/inpatient_all_final_1.csv'
# df1 = pd.read_csv(path).iloc[:,1:]
# df1.drop(columns = ['Veteran flag','Event date','Marital status', 'Marital status encoded',
#                     'State','Ruca category'], inplace=True, axis=1)
# X_admission1 = df1.drop(columns = ['Readmission', 'Died'])
# Y_admission1 = df1[['Readmission']]

# Split Train and Test
# X_train_ad1, X_test_ad1, y_train_ad1, y_test_ad1 = train_test_split(X_admission1, Y_admission1, test_size=0.20, random_state=42)

# 'Internalpatientid' is
# Transform Data

# Import Data
path = '/home/daisy/FDA_Dataset/inpatient_all_final_1.csv'
df1 = pd.read_csv(path).iloc[:,1:]
X_admission1 = df1.drop(columns = ['readmission within 300 days', 'died_within_900days'])
Y_admission1 = df1[['readmission within 300 days']]

X_mortality1 = df1.drop(columns = ['died_within_900days'])
Y_mortality1 = df1[['died_within_900days']]

# Split Train and Test
X_train_ad1, X_test_ad1, y_train_ad1, y_test_ad1 = train_test_split(X_admission1, Y_admission1, test_size=0.20, random_state=42)


transform_steps = [("ImputeNumeric", ImputeNumeric()),
                   ('RemoveSkewnessKurtosis', RemoveSkewnessKurtosis()),
                   ('StandardizeStandardScaler', Standardize(RobustScaler()))]
transform_pipeline = Pipeline(transform_steps)

data_prepared = transform_pipeline.transform(X_train_ad1)

/home/vivi/FDA/src/models/transformation.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[i] = self.scalar.fit_transform(X[i].values.reshape(-1,1))


In [3]:
data_prepared.columns

Index(['Internalpatientid', 'num_transfers_log_rob_scaled',
       'Age 100-120 hypertension_log_rob_scaled',
       'Age 40-60 hypotension_log_rob_scaled', 'OTHERS_log_rob_scaled',
       'stay_std_log_rob_scaled', 'total_procedure_log_rob_scaled',
       'Age 20-40 healthy_log_rob_scaled', 'Medical_log_rob_scaled',
       'unique_admitting_specialty_log_rob_scaled',
       'Total medications_log_rob_scaled', 'Others_Specialty_log_rob_scaled',
       'Age 80-100 hypotension_log_rob_scaled',
       'num_cvd_readmission_log_rob_scaled', 'Age 80-100_log_rob_scaled',
       'specialty medical count_log_rob_scaled',
       'Num med per admission mean_log_rob_scaled',
       'Age 60-80 hypertension_log_rob_scaled',
       'Age 100-120 hypotension_log_rob_scaled', 'period std_log_rob_scaled',
       'Age 60-80_log_rob_scaled', 'SURGERY_log_rob_scaled',
       'MEDICINE_log_rob_scaled', 'freq_log_rob_scaled',
       'Age 100-120 healthy_log_rob_scaled', 'Age 40-60_log_rob_scaled',
       'Age

In [2]:
# Grid Search CV
import numpy as np
import pandas as pd
import scipy as sp
import copy,os,sys,psutil
import lightgbm as lgb
from lightgbm.sklearn import LGBMRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import dump_svmlight_file
from sklearn.linear_model import LogisticRegression
 
from sklearn import metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
 
def print_best_score(gsearch,param_test):
    print("Best score: %0.3f" % gsearch.best_score_)
    print("Best parameters set:")
    for param_name in sorted(param_test.keys()):
        print("\t%s: %r" % (param_name,gsearch.best_params_[param_name]))
 
LogisticRegression_param = { 
    'penalty' : ['l1','l2']
    }

def LogisticRegression_Grid_CV(X_train,y_train, LogisticRegression_param):
    estimator = LogisticRegression()
    gsearch = GridSearchCV(estimator , param_grid = LogisticRegression_param, scoring='roc_auc', cv=5)
    gsearch.fit(X_train, y_train)
    #gsearch.grid_scores_, gsearch.best_params_, gsearch.best_score_
    return (gsearch,LogisticRegression_param)

gsearch,LogisticRegression_param =LogisticRegression_Grid_CV(data_prepared, y_train_ad1, LogisticRegression_param)

/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STO

In [3]:
gsearch.best_params_

LogisticRegression()

In [ ]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument("--isdev", help="run to test pipeline: 1 - True, 0 - False", type=int)
    parser.add_argument("--model_type", help="select model architecture", type=str)

    args = parser.parse_args()

    print(f"Selected model type is: {args.model_type}")

    target_value, covariate_value, target_codename = get_target_variates(args) ## get covariates independent of iscovariates True or False

    experiment_config = {
        "input_length":360,
        "prediction_front":180,
        "prediction_back":0,
        "batch_size":512, # 1024
        "max_n_epochs":1 if is_developer else 1000,
        "nr_epochs_val_period":1,
        "data_path":'/data/public/MLA/share/',
        "dataset_name":'141_forecasting_Vitals_v0.2.pickle',

        "target_value":target_value,
        "covariate_value": covariate_value,
        
        "early_stopping_kwargs":{"monitor":'val_loss', "min_delta":0.0001, "patience":1 if is_developer else 15, "verbose":True},
        "model_type": args.model_type, ## Important
        "task_type":f"{target_codename}_hpo_{args.target}", 
        "gpu_num":[args.gpu], # or [1]
        
        "iscovariates": iscovariates,
        # "model_parameters": params, ## FALSE for HPO config
        "save_path":'experiment_logs', ## don't change for consistency
    }

    if iscovariates:
        print(f"Covariates selected for training: {covariate_value}\n")
        training, validation, testing, co_training, co_validation, co_testing = build_training_dataset(experiment_config=experiment_config)
        covariate_list = [co_training, co_validation, co_testing]

    else:
        print("No covariates selected for training!")
        training, validation, testing = build_training_dataset(experiment_config=experiment_config)
        covariate_list = None


    timestr = time.strftime("%Y%m%d_%H%M%S")
    
    if covariate_list is not None:
        model_hpo_name = f"{timestr}_{args.model_type}_wc_{target_codename}_hpo_{args.target}"
    else:
        model_hpo_name = f"{timestr}_{args.model_type}_woc_{target_codename}_hpo_{args.target}"

    ## print model_hpo_name for reference
    print(model_hpo_name)

    if args.model_type=='nhits':
        optuna_objective = lambda trial: objective_nhits(trial, training, validation, covariate_list, model_hpo_name, experiment_config['gpu_num'], experiment_config=experiment_config)

    elif args.model_type=='tft':
        optuna_objective = lambda trial: objective_tft(trial, training, validation, covariate_list, model_hpo_name, experiment_config['gpu_num'], experiment_config=experiment_config)
    
    elif args.model_type=='nbeats':
        experiment_config['generic_architecture'] = True ## for generic model
        optuna_objective = lambda trial: objective_nbeats(trial, training, validation, covariate_list, model_hpo_name, experiment_config['gpu_num'], experiment_config=experiment_config)

    elif args.model_type=='nbeats_I':
        experiment_config['generic_architecture'] = False ## for interpretibility model
        optuna_objective = lambda trial: objective_nbeats_I(trial, training, validation, covariate_list, model_hpo_name, experiment_config['gpu_num'], experiment_config=experiment_config)

    elif args.model_type=='tcn':
        optuna_objective = lambda trial: objective_tcn(trial, training, validation, covariate_list, model_hpo_name, experiment_config['gpu_num'], experiment_config=experiment_config)
    
    elif args.model_type=='deepar':
        optuna_objective = lambda trial: objective_deepar(trial, training, validation, covariate_list, model_hpo_name, experiment_config['gpu_num'], experiment_config=experiment_config)

    else: raise NotImplementedError

    study = optuna.create_study(direction='minimize')
    study.optimize(optuna_objective, n_trials=TRIALS, callbacks=[print_callback])        

    print(f"\n\nBest Hyperparameters for Target: {target_value} and {args.model_type} are:")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params} \n\n")

    params = copy.deepcopy(study.best_trial.params)

    experiment_config["model_parameters"] = params
    experiment_config["task_type"] = f"{target_codename}_bprm_{args.target}" ## Important (fm: filter minmax, rw: raw, fs: filter std.)
    experiment_config['current_time'] = timestr ## keep timestamp consistent with hpo and best model training. easy to follow in future.
    
    execute_experiment(training, validation, testing, covariate_list, experiment_config=experiment_config)